In [1]:
import torch
from transformers import *

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


In [2]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

100%|█████████████████████████████████████████████████████████████████| 267967963/267967963 [11:36<00:00, 384942.88B/s]


In [47]:
lm = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

In [51]:
input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])
input_ids = input_ids.to("cuda")

In [50]:
model.eval()
lm.eval()

DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (dropout): Dropout(p=0.1, inplace=False)
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): 

In [52]:
model.to("cuda")
lm.to("cuda")

DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (dropout): Dropout(p=0.1, inplace=False)
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): 

In [53]:
with torch.no_grad():
        model_states = model(input_ids)
        lm_states = lm(input_ids)

In [54]:
len(model_states[0][0][-1])

768

In [55]:
len(lm_states[0][0][-1])

30522

In [59]:
predicted_index = torch.argmax(lm_states[0][0, -1, :]).item()
predicted_index

1012

In [62]:
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
predicted_token

['.']

In [63]:
torch.topk(lm_states[0][0, -1, :], 5)

torch.return_types.topk(
values=tensor([19.8478,  9.8374,  7.5366,  7.4306,  6.6368], device='cuda:0'),
indices=tensor([1012, 1010, 1029, 1007, 1996], device='cuda:0'))

In [73]:
p = torch.topk(torch.softmax(lm_states[0][0][-1], -1, torch.float32), 5)
p

torch.return_types.topk(
values=tensor([9.9993e-01, 4.4927e-05, 4.5007e-06, 4.0482e-06, 1.8303e-06],
       device='cuda:0'),
indices=tensor([1012, 1010, 1029, 1007, 1996], device='cuda:0'))

In [75]:
p.values[0].item()

0.9999347925186157

In [76]:
p.indices[0].item()

1012

In [78]:
tokenizer.convert_ids_to_tokens([1012])

['.']